In [ ]:
from datetime import datetime, timezone
import MetaTrader5 as mt5


In [ ]:
import os
import time
import pandas as pd
import pprint
from plotly.subplots import make_subplots
import plotly.graph_objects as go


In [ ]:
# connect to MetaTrader 5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()


In [ ]:
# request connection status and parameters
pprint.pprint(mt5.terminal_info()._asdict())
# get data on MetaTrader 5 version
print(mt5.version())


In [ ]:
# connect to the trade account specifying a server
authorized = mt5.login(122334, server="Exness-MT5Trial7")
if authorized:
    account_info = mt5.account_info()
    if account_info != None:
        account_info_dict = account_info._asdict()
        # convert the dictionary into DataFrame and print
        df = pd.DataFrame(list(account_info_dict.items()), columns=["property", "value"])
        print(df)
else:
    print("failed to connect to trade account, error code =", mt5.last_error())


In [ ]:
# get bars
import calendar

symbols = ["XAUUSDm", "GBPUSDm", "EURUSDm", "EURCADm"]
interval = "15m"
tf = mt5.TIMEFRAME_M15
months = [1, 2, 3, 4, 5]
year = 2023

for symbol in symbols:
    for month in months:
        from_date = datetime(year, month, 1).replace(tzinfo=timezone.utc).timestamp()
        to_date = datetime(year, month, calendar.monthrange(year, month)[1]).replace(tzinfo=timezone.utc).timestamp()

        df = mt5.copy_rates_range(symbol, tf, from_date, to_date)
        df = pd.DataFrame(df)
        df["time"] += -time.timezone
        df["time"] = pd.to_datetime(df["time"], unit="s")
        print("{}, from: {} to: {}".format(symbol, df.iloc[0]["time"], df.iloc[-1]["time"]))

        df.columns = ["Open time", "Open", "High", "Low", "Close", "Volume", "Spread", "Real_Volume"]
        df = df[["Open time", "Open", "High", "Low", "Close", "Volume"]]
        filename = "{}-{}-{}-{:02d}".format(symbol, interval, year, month)
        df.to_csv(r"G:\AI\MT5\{}.csv".format(filename), index=0)